In [5]:
import os
import sys
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,isnan,when,sum,lit
from pyspark.sql.functions import corr
import itertools


In [6]:
def init_spark():
    spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()
    return spark

In [3]:
spark_object = init_spark()
train_identity = spark_object.read.option("delimiter", ",").csv("./data/train_identity.csv", header='true', inferSchema='true')
train_transaction = spark_object.read.option("delimiter", ",").csv("./data/train_transaction.csv", header='true', inferSchema='true')

23/03/14 08:29:39 WARN Utils: Your hostname, karyubuntu resolves to a loopback address: 127.0.1.1; using 192.168.0.165 instead (on interface wlp0s20f3)
23/03/14 08:29:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/14 08:29:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [22]:
print(train_identity.count())
print(train_transaction.count())
print(train_identity.head())
print(train_transaction.head())

144233
590540
23/03/13 23:52:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
Row(TransactionID=2987004, id_01=0.0, id_02=70787.0, id_03=None, id_04=None, id_05=None, id_06=None, id_07=None, id_08=None, id_09=None, id_10=None, id_11=100.0, id_12='NotFound', id_13=None, id_14=-480.0, id_15='New', id_16='NotFound', id_17=166.0, id_18=None, id_19=542.0, id_20=144.0, id_21=None, id_22=None, id_23=None, id_24=None, id_25=None, id_26=None, id_27=None, id_28='New', id_29='NotFound', id_30='Android 7.0', id_31='samsung browser 6.2', id_32=32.0, id_33='2220x1080', id_34='match_status:2', id_35='T', id_36='F', id_37='T', id_38='T', DeviceType='mobile', DeviceInfo='SAMSUNG SM-G892A Build/NRD90M')
Row(TransactionID=2987000, isFraud=0, TransactionDT=86400, TransactionAmt=68.5, ProductCD='W', card1=13926, card2=None, card3=150.0, card4='discover', card5=142.0, card6='credit', addr1=315

In [23]:
train = train_transaction.join(train_identity, on="TransactionID", how='inner')

In [31]:
print(train.count())
train.head()

144233


Row(TransactionID=2987048, isFraud=0, TransactionDT=87317, TransactionAmt=42.294, ProductCD='C', card1=15885, card2=545.0, card3=185.0, card4='visa', card5=138.0, card6='debit', addr1=None, addr2=None, dist1=None, dist2=None, P_emaildomain='outlook.com', R_emaildomain='outlook.com', C1=1.0, C2=1.0, C3=0.0, C4=1.0, C5=0.0, C6=1.0, C7=1.0, C8=1.0, C9=0.0, C10=1.0, C11=1.0, C12=1.0, C13=1.0, C14=1.0, D1=0.0, D2=None, D3=None, D4=0.0, D5=None, D6=0.0, D7=None, D8=None, D9=None, D10=0.0, D11=None, D12=0.0, D13=0.0, D14=0.0, D15=0.0, M1=None, M2=None, M3=None, M4='M2', M5=None, M6=None, M7=None, M8=None, M9=None, V1=None, V2=None, V3=None, V4=None, V5=None, V6=None, V7=None, V8=None, V9=None, V10=None, V11=None, V12=0.0, V13=0.0, V14=1.0, V15=1.0, V16=1.0, V17=1.0, V18=1.0, V19=1.0, V20=1.0, V21=1.0, V22=1.0, V23=1.0, V24=1.0, V25=1.0, V26=1.0, V27=0.0, V28=0.0, V29=0.0, V30=0.0, V31=1.0, V32=1.0, V33=1.0, V34=1.0, V35=0.0, V36=0.0, V37=1.0, V38=1.0, V39=1.0, V40=1.0, V41=1.0, V42=1.0, V43=1

In [7]:
total_count = train.count()
def null_percentage(df, column):
    null_count = df.where(col(column).isNull()).count()
    return null_count / total_count

 
for column in train.columns:
    if null_percentage(train, column) > 0.05:
        train = train.drop(col(column))

In [8]:
print((train.count(), len(train.columns)))
train.toPandas().to_csv('./data/traincleaned.csv')

(144233, 194)


In [9]:
# load data from train cleaned file 

In [7]:
train = spark_object.read.option("delimiter", ",").csv("data/train_combined.csv", header='true', inferSchema='true')
test = spark_object.read.option("delimiter", ",").csv("data/test_combined.csv", header='true', inferSchema='true')

print((train.count(),len(train.columns)))
print((test.count(),len(test.columns)))

In [21]:
# def top_missing_cols(train, n=None, thresh=80):
#     # Convert the DataFrame to Pandas and process it
    
    
#     dff = dff.toPandas()
#     dff = dff.melt(var_name='col', value_name='missing_percent')
#     dff = dff.sort_values(by=['missing_percent'], ascending=False).reset_index(drop=True)

#     print(f'There are {total_cols} columns in this dataset.')
#     print(f'There are {dff[dff["missing_percent"] > thresh].shape[0]} columns with missing percent values than {thresh}%')
    
#     # Return the top missing columns
#     if n:
#         return dff.head(n)
#     else:
#         return dff






In [22]:

# df_missing = top_missing_cols(train, n=None, thresh=50)
# missing_cols = df_missing['col']

C:\Program Files\Apache Spark\spark-3.1.2-bin-hadoop2.7\spark-3.1.2-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
C:\Program Files\Apache Spark\spark-3.1.2-bin-hadoop2.7\spark-3.1.2-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
C:\Program Files\Apache Spark\spark-3.1.2-bin-hadoop2.7\spark-3.1.2-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:186: Performance

There are 434 columns in this dataset.
There are 214 columns with missing percent values than 50%


C:\Program Files\Apache Spark\spark-3.1.2-bin-hadoop2.7\spark-3.1.2-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
C:\Program Files\Apache Spark\spark-3.1.2-bin-hadoop2.7\spark-3.1.2-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
C:\Program Files\Apache Spark\spark-3.1.2-bin-hadoop2.7\spark-3.1.2-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:186: Performance

In [28]:
# nan_dict = {}
# for col_name in missing_cols:
#     count_val = train.select([count(when(col(c).isNull(), c)).alias(c) for c in [col_name]]).collect()[0][0]
#     try:
#         nan_dict[count_val].append(col_name)
#     except:
#         nan_dict[count_val] = [col_name]

# for k,v in nan_dict.items():
#     print(f'#####' * 4)
#     print(f'NAN count = {k} percent: {(int(k)/train.count())*100} %')
#     print(v)

####################
NAN count = 585793 percent: 99.19615944728554 %
['id_24']
####################
NAN count = 585408 percent: 99.13096487960172 %
['id_25']
####################
NAN count = 585385 percent: 99.12707013919464 %
['id_07', 'id_08']
####################
NAN count = 585381 percent: 99.12639279303687 %
['id_21']
####################
NAN count = 585377 percent: 99.12571544687913 %
['id_26']
####################
NAN count = 585371 percent: 99.1246994276425 %
['id_27', 'id_23', 'id_22']
####################
NAN count = 552913 percent: 93.62837403054831 %
['dist2']
####################
NAN count = 551623 percent: 93.40992989467267 %
['D7']
####################
NAN count = 545427 percent: 92.36072069631184 %
['id_18']
####################
NAN count = 528588 percent: 89.50926270870728 %
['D13']
####################
NAN count = 528353 percent: 89.46946862193924 %
['D14']
####################
NAN count = 525823 percent: 89.04104717715988 %
['D12']
####################
NAN count = 52

In [8]:
cat_cols = (['ProductCD'] +
            ['card{}'.format(i) for i in range(1, 7)] +
            ['addr1', 'addr2', 'P_emaildomain', 'R_emaildomain'] +
            ['M{}'.format(i) for i in range(1, 10)] +
            ['DeviceType', 'DeviceInfo'] +
            ['id_{}'.format(i) for i in range(12, 39)])

type_map = {c: 'string' for c in cat_cols}

for c in cat_cols:
    train = train.withColumn(c, col(c).cast('string'))
    test = test.withColumn(c, col(c).cast('string'))

In [9]:

id_cols = ['TransactionID', 'TransactionDT']
target = 'isFraud'

numeric_cols =  [
    'TransactionAmt', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 
    'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 
    'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 
    'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 
    'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 
    'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 
    'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 
    'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 
    'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 
    'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 
    'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 
    'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 
    'V133', 'V134', 'V135', 'V136', 'V137', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144', 
    'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V156', 
    'V157', 'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 
    'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V180', 
    'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 
    'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 
    'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 
    'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 
    'V229', 'V230', 'V231', 'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V240', 
    'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V250', 'V251', 'V252', 
    'V253', 'V254', 'V255', 'V256', 'V257', 'V258', 'V259', 'V260', 'V261', 'V262', 'V263', 'V264', 
    'V265', 'V266', 'V267', 'V268', 'V269', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 
    'V277', 'V278', 'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 
    'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 
    'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 
    'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321', 'V322', 'V323', 'V324', 
    'V325', 'V326', 'V327', 'V328', 'V329', 'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 
    'V337', 'V338', 'V339', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 
    'id_09', 'id_10', 'id_11'
]


reduced_vcols = ['V1', 'V3', 'V4', 'V6', 'V8', 'V11', 'V13', 'V14', 'V17', 'V20', 
 'V23', 'V26', 'V27', 'V30', 'V36', 'V37', 'V40', 'V41', 'V44', 'V47', 'V48', 'V54', 'V56', 'V59', 
 'V62', 'V65', 'V67', 'V68', 'V70', 'V76', 'V78', 'V80', 'V82', 'V86', 'V88', 'V89', 'V91', 'V96', 
 'V98', 'V99', 'V104', 'V107', 'V108', 'V111', 'V115', 'V117', 'V120', 'V121', 'V123', 'V124', 'V127', 
 'V129', 'V130', 'V136', 'V138', 'V139', 'V142', 'V147', 'V156', 'V162', 'V165', 'V160', 'V166', 'V178',
 'V176', 'V173', 'V182', 'V187', 'V203', 'V205', 'V207', 'V215', 'V169', 'V171', 'V175', 'V180', 'V185', 
 'V188', 'V198', 'V210', 'V209', 'V218', 'V223', 'V224', 'V226', 'V228', 'V229', 'V235', 'V240', 'V258', 
 'V257', 'V253', 'V252', 'V260', 'V261', 'V264', 'V266', 'V267', 'V274', 'V277', 'V220', 'V221', 'V234', 
 'V238', 'V250', 'V271', 'V294', 'V284', 'V285', 'V286', 'V291',
 'V297', 'V303', 'V305', 'V307', 'V309', 'V310', 'V320', 'V281', 'V283', 'V289', 'V296', 'V301', 'V314', 'V332', 'V325', 'V335', 'V338']

In [10]:
drop_cols = [col for col in train.columns if col.startswith('V') and col not in reduced_vcols]

print(f'dropping {len(drop_cols)} columns')
df_train = train.drop(*drop_cols)
df_test = test.drop(*drop_cols)

dropping 211 columns


In [11]:
y_train = df_train.select(col('isFraud'))
X_train = df_train.select([col for col in df_train.columns if col != 'isFraud'])
X_test = df_test.select([col for col in df_test.columns])

# print(X_train.count(), len(X_train.columns))
# print(X_test.count(), len(X_test.columns))


In [12]:
from pyspark.ml.feature import StringIndexer
for col in X_train.columns:
    if col in cat_cols:
        indexer = StringIndexer(inputCol=col, outputCol=col+"_indexed")
        indexer.setHandleInvalid("keep")
        model = indexer.fit(X_train)
        X_train = model.transform(X_train).drop(col)
        X_train = X_train.withColumnRenamed(col+"_indexed", col)

        X_test = model.transform(X_test).drop(col)
        X_test = X_test.withColumnRenamed(col+"_indexed", col)

In [13]:
rem_cols = []
rem_cols.extend(['TransactionDT','TransactionID'])

cols = [col for col in X_train.columns if col not in rem_cols]
len(cols)

220

In [11]:
print(X_train.printSchema())

root
 |-- TransactionID: integer (nullable = true)
 |-- TransactionDT: integer (nullable = true)
 |-- TransactionAmt: double (nullable = true)
 |-- dist1: double (nullable = true)
 |-- dist2: double (nullable = true)
 |-- C1: double (nullable = true)
 |-- C2: double (nullable = true)
 |-- C3: double (nullable = true)
 |-- C4: double (nullable = true)
 |-- C5: double (nullable = true)
 |-- C6: double (nullable = true)
 |-- C7: double (nullable = true)
 |-- C8: double (nullable = true)
 |-- C9: double (nullable = true)
 |-- C10: double (nullable = true)
 |-- C11: double (nullable = true)
 |-- C12: double (nullable = true)
 |-- C13: double (nullable = true)
 |-- C14: double (nullable = true)
 |-- D1: double (nullable = true)
 |-- D2: double (nullable = true)
 |-- D3: double (nullable = true)
 |-- D4: double (nullable = true)
 |-- D5: double (nullable = true)
 |-- D6: double (nullable = true)
 |-- D7: double (nullable = true)
 |-- D8: double (nullable = true)
 |-- D9: double (nullable = tr

In [18]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
# Define a list of column names that need to be converted
convert_cols = [col_name1 for col_name1, data_type in X_train.dtypes if data_type in ["integer", "double"]]
print(convert_cols)
# Loop through the list of columns to apply the VectorAssembler
for col_name2 in convert_cols:
    assembler = VectorAssembler(inputCols=[col_name2], outputCol=col_name2+"_vector",handleInvalid="skip")
    X_train = assembler.transform(X_train)
    X_test = assembler.transform(X_test)
    X_train = X_train.drop(col_name2)
    X_test = X_test.drop(col_name2)

all_cols = X_train.columns
vector_cols = [c for c in all_cols if "_vector" in c]
for c in vector_cols:
    new_name = c.replace("_vector","")
    X_train = X_train.withColumnRenamed(c,new_name)
    X_test = X_test.withColumnRenamed(c,new_name)
# Print the schema of the converted DataFrame
# print(X_train.printSchema())
print(X_test.printSchema())

[]
root
 |-- TransactionID: integer (nullable = true)
 |-- TransactionDT: integer (nullable = true)
 |-- TransactionAmt: vector (nullable = true)
 |-- dist1: vector (nullable = true)
 |-- dist2: vector (nullable = true)
 |-- C1: vector (nullable = true)
 |-- C2: vector (nullable = true)
 |-- C3: vector (nullable = true)
 |-- C4: vector (nullable = true)
 |-- C5: vector (nullable = true)
 |-- C6: vector (nullable = true)
 |-- C7: vector (nullable = true)
 |-- C8: vector (nullable = true)
 |-- C9: vector (nullable = true)
 |-- C10: vector (nullable = true)
 |-- C11: vector (nullable = true)
 |-- C12: vector (nullable = true)
 |-- C13: vector (nullable = true)
 |-- C14: vector (nullable = true)
 |-- D1: vector (nullable = true)
 |-- D2: vector (nullable = true)
 |-- D3: vector (nullable = true)
 |-- D4: vector (nullable = true)
 |-- D5: vector (nullable = true)
 |-- D6: vector (nullable = true)
 |-- D7: vector (nullable = true)
 |-- D8: vector (nullable = true)
 |-- D9: vector (nullable =

In [16]:
from pyspark.ml.feature import MinMaxScaler

for col in cols:
    if col not in cat_cols:
        # concatenate train and test data
        dff = X_train.select(col).union(X_test.select(col))

        # create scaler and fit to data
        scaler = MinMaxScaler(inputCol=col, outputCol=col+"_scaled")
        scaler_model = scaler.fit(dff)

        # transform train and test data using fitted scaler
        train = scaler_model.transform(X_train)
        test = scaler_model.transform(X_test)

        # rename the newly created scaled columns
        train = X_train.withColumnRenamed(col+"_scaled", col)
        test = X_test.withColumnRenamed(col+"_scaled", col)

Py4JJavaError: An error occurred while calling o13259.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 115.0 failed 1 times, most recent failure: Lost task 0.0 in stage 115.0 (TID 1265) (192.168.0.155 executor driver): java.lang.IllegalArgumentException: requirement failed: Nothing has been added to this summarizer.
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.stat.SummarizerBuffer.max(Summarizer.scala:710)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.$anonfun$eval$1(Summarizer.scala:361)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map